We used a modification of Ripser++ that includes additional optimization for faster (up to 55%) computation of RTD barcodes. Normal Ripser++ can also be used instead from our modification.

In [ ]:
!pip install git+https://github.com/ArGintum/RipserZeros.git

If you don't have Transformers package installed, please do it now --- it is  required to work with BERT

In [ ]:
#!pip install transformers

In [4]:
import itertools
import re
import subprocess
import json
import torch

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import defaultdict
from tqdm.notebook import tqdm
from transformers import BertTokenizer, BertModel, BertConfig, BertForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, XLMRobertaConfig

import ripserplusplus as rpp_py

# Fix random seed for reproduciability 
np.random.seed(42)

# 1. Computing RTD scores on BLIMP data

Download pretrained BERT model

In [5]:
device='cuda'
model_path = tokenizer_path = "bert-base-cased"

config = BertConfig.from_pretrained('bert-base-cased', output_hidden_states=False, output_attentions=True)
model = BertForSequenceClassification.from_pretrained(model_path, config=config).to(device)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=False)

MAX_LEN = max_tokens_amount = 128

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Set dataset structure: dictionary of pairs `category name' : list of filenames containing data about phenomenons of that category

In [6]:
blimp_structure = {
    'Anaphor agreement' : ['anaphor_gender_agreement.json', 'anaphor_number_agreement.json'],
    'Argument structure' : ['animate_subject_passive.json', 'animate_subject_trans.json', 'causative.json', 'drop_argument.json', 'inchoative.json', 'intransitive.json', 'passive_1.json', 'passive_2.json', 'transitive.json'],
    'Binding' : ['principle_A_c_command.json', 'principle_A_case_1.json', 'principle_A_case_2.json', 'principle_A_domain_1.json', 'principle_A_domain_2.json', 'principle_A_domain_3.json', 'principle_A_reconstruction.json'],
    'Control / raising' : ['existential_there_object_raising.json', 'existential_there_subject_raising.json', 'expletive_it_object_raising.json', 'tough_vs_raising_1.json', 'tough_vs_raising_2.json'],
    'Determiner-Noun agr.' : ['determiner_noun_agreement_1.json', 'determiner_noun_agreement_2.json', 'determiner_noun_agreement_irregular_1.json', 'determiner_noun_agreement_irregular_2.json', 
                              'determiner_noun_agreement_with_adjective_1.json', 'determiner_noun_agreement_with_adj_2.json', 'determiner_noun_agreement_with_adj_irregular_1.json', 
                              'determiner_noun_agreement_with_adj_irregular_2.json'],
    'Ellipsis' : ['ellipsis_n_bar_1.json', 'ellipsis_n_bar_2.json'],
    'Filler gap' : ['wh_questions_object_gap.json', 'wh_questions_subject_gap.json', 'wh_questions_subject_gap_long_distance.json', 'wh_vs_that_no_gap.json', 'wh_vs_that_no_gap_long_distance.json',
                    'wh_vs_that_with_gap.json', 'wh_vs_that_with_gap_long_distance.json'],
    'Irregular forms' : ['irregular_past_participle_adjectives.json', 'irregular_past_participle_verbs.json'],
    'Island effects' : ['adjunct_island.json', 'complex_NP_island.json', 'coordinate_structure_constraint_complex_left_branch.json', 'coordinate_structure_constraint_object_extraction.json',
                        'left_branch_island_echo_question.json', 'left_branch_island_simple_question.json', 'sentential_subject_island.json', 'wh_island.json'],
    'NPI licensing' : ['matrix_question_npi_licensor_present.json', 'npi_present_1.json', 'npi_present_2.json', 'only_npi_licensor_present.json', 'only_npi_scope.json', 'sentential_negation_npi_licensor_present.json',
                       'sentential_negation_npi_scope.json'],
    'Quantifiers' : ['existential_there_quantifiers_1.json', 'existential_there_quantifiers_2.json', 'superlative_quantifiers_1.json', 'superlative_quantifiers_2.json'],
    'Subject-verb agr.' : ['distractor_agreement_relational_noun.json', 'distractor_agreement_relative_clause.json', 'irregular_plural_subject_verb_agreement_1.json', 'irregular_plural_subject_verb_agreement_2.json',
                           'regular_plural_subject_verb_agreement_1.json', 'regular_plural_subject_verb_agreement_2.json']
}

Function for computing RTD of two graphs given as adjacency matrices


In [45]:
def rtd(a, b, dim=1, mode='max'):
  # Parameters:
  #           a    - adjacency matrix (symmetric) of the first graph
  #           b    - adjacency matrix (symmetric) of the first graph
  #           dim  - dimension of RTD. Default: 1 (in all our experiments we used this value only)
  #           mode - variant of RTD to use. Default: 'max' (in all our experiments we used this value only)

  # Returns: list of bars of RTD(<a>, <b>) in dimension <dim>

  if mode != 'max' and mode != 'min':
     raise ValueError("Wrong Value for 'mode' parameter. RTD mode can be either 'min' or 'max' ")

  n = a.shape[0]
  d = np.zeros((2 * n, 2 * n))
  
  if mode == 'max':
    d[n:, :n] = torch.maximum(a.cpu(), b.cpu()).detach().numpy()  
    d[:n, n:] = torch.maximum(a.cpu(), b.cpu()).detach().numpy()  
    d[n:, n:] = b.cpu().detach().numpy()
  else:
    d[n:, :n] = a.cpu().detach().numpy()
    d[:n, n:] = a.cpu().detach().numpy()
    d[n:, n:] = torch.minimum(a.cpu(), b.cpu()).detach().numpy() 

  m = d[n:, :n].mean()
  d[d < m*(1e-6)] = 0
  results = rpp_py.run("--format distance --mode rtd --dim " + str(dim), d)['dgms'][dim]
  return results

Main function for scoring Rules on individual heads

In [56]:
def attentive_work_in_class(model, phenomenons, root_dir="", metric="rtd", finish_pos=1000, skip=0, attention_layers=12, attention_heads=12):
  # Parameters:
  #           model       - linguistic model for scoring
  #           phenomenons - dictionary structure of the dataset
  #           root_dir    - path to the directory with data files
  #           metric      - topology metric to be used for scoring. Can be either 'rtd' for RTD or 'h0s' for H0_s.
  #           finish_pos   
  #           skip        - last two arguments gives the range of strings from data files to use : [skip; finish_pos)
  #                         Default: skip = 0, finish_pos = 1000 --- this gives all strings from original BLIMP files

  # Returns: matrix of size (finish_pos - slip)*<total number of phenomenons> X <total number of heads>

  if metric != 'rtd' and metric != 'h0s':
     raise ValueError("Wrong Value for 'metric' parameter. Metric to be used can be either 'rtd' or 'h0s' ")

  samples = finish_pos - skip
  full_matrix = np.zeros((67 * samples, attention_layers * attention_heads))
  pos = 0

  model.eval()

  for name in phenomenons:
    for json_name in phenomenons[name]:
      step_cnt = 0  

      fle = open(root_dir + json_name + 'l', 'r')
 
      for s in fle.readlines():
        if step_cnt < skip:
          step_cnt += 1
          continue

        if step_cnt >= finish_pos:
          break

        data = json.loads(s)
        inputs = tokenizer(data['sentence_good'], max_length=128, return_tensors="pt").to(device)
        inputs2 = tokenizer(data['sentence_bad'], max_length=128, return_tensors="pt").to(device)

        step_cnt += 1

        # To ensure one-to-one correspondence between tokens when sequences are unequal we take only the `tail' from the longest
        n_len = min(len(inputs['input_ids'][0]), len(inputs2['input_ids'][0]))
        goods_list = torch.stack(model(**inputs)[1], dim=0)[:attention_layers, 0, :attention_heads, -n_len:, -n_len:]
        bads_list = torch.stack(model(**inputs2)[1], dim=0)[:attention_layers, 0, :attention_heads, -n_len:, -n_len:]
        # ---------------------------------------

        for i in range(attention_layers):
          for j in range(attention_heads):       
            # First, we symmetrize attention matrices
            d_good = 1.0 - (torch.triu(goods_list[i][j], 1) + torch.transpose(torch.triu(goods_list[i][j], 1), 0, 1))
            d_good -= torch.diag(torch.diag(d_good))
            d_bad = 1.0 - (torch.triu(bads_list[i][j], 1) + torch.transpose(torch.triu(bads_list[i][j], 1), 0, 1))
            d_bad -= torch.diag(torch.diag(d_bad)) 
            # ---------------------------------------

            if metric == "rtd":
              results = rtd(d_good, d_bad, mode='max')
            else:
              results = rpp_py.run("--format distance --mode original --dim 0", d_good)['dgms'][0]

            if len(results) > 0:
              term_left = np.mean([tmp_x[1] - tmp_x[0] for tmp_x in results])
            else:
              term_left = 0.0
            
            if metric == "rtd":
              results = rtd(d_bad, d_good, mode='max')
            else:
              results = rpp_py.run("--format distance --mode original --dim 0", d_bad)['dgms'][0]

            if len(results) > 0:
              term_right = np.mean([tmp_x[1] - tmp_x[0] for tmp_x in results])
            else:
              term_right = 0.0

            # If Rule I gives the correct result for this pair, corresponding element of the matrix is set to 1
            # Otherwise, Rule II is correct (since for A != B holds RTD(A, B) != RTD(B, A) ) and full_matrix[i, j] set to 0
            if term_left < term_right:
              full_matrix[pos, attention_heads * i + j] = 1
            # ---------------------------------------
        pos += 1

      fle.close()
      
      smol_floppa = np.mean(full_matrix[pos - samples : pos, :], axis=0)
      arg = np.argmax(smol_floppa)
      arg2 = np.argmin(smol_floppa)
    
      print('phenomenon ', json_name[:-5], 'Rule I: best head (', arg // attention_heads, ',', arg % attention_heads, ') acc.: ', 100 * np.max(smol_floppa),  '/// Rule II: best head (', arg2 // attention_heads, ',', arg2 % attention_heads, ') acc.: ', 100 * (1 - np.min(smol_floppa)) )
    
    big_floppa = np.mean(full_matrix[pos - samples * len(phenomenons[name]) : pos, :], axis=0)
    arg = np.argmax(big_floppa)
    arg2 = np.argmin(big_floppa)
    
    print('category ', name, 'Rule I: best head (', arg // attention_heads, ',', arg % attention_heads, ') acc.: ', 100 * np.max(big_floppa),  '/// Rule II: best head (', arg2 // attention_heads, ',', arg2 % attention_heads, ') acc.: ', 100 * (1 - np.min(big_floppa)) )
    print('\n')
    
  
  total_floppa = np.mean(full_matrix, axis=0)
  arg = np.argmax(total_floppa)
  arg2 = np.argmin(total_floppa)
  print('Total. Rule I: best head (', arg // attention_heads, ',', arg % attention_heads, ') acc.: ', 100 * np.max(total_floppa),  '/// Rule II: best head (', arg2 // attention_heads, ',', arg2 % attention_heads, ') acc.: ', 100 * (1 - np.min(total_floppa)) )

  return full_matrix

In [ ]:
fullBERT_rtd_matrix = attentive_work_in_class(model, blimp_structure, root_dir="data/BLIMP/", metric="rtd", finish_pos=10)
#np.save('fullBERT_rtd.dat', fullBERT_rtd_matrix)

The same but using H0_S:

In [ ]:
fullBERT_h0s_matrix = attentive_work_in_class(model, blimp_structure, root_dir="data/BLIMP/", metric="h0s")
#np.save('fullBERT_h0s.dat', fullBERT_rtd_matrix)

Compute scores on generated auxillary data to perform ensemble selection

In [ ]:
smallBERT_rtd_matrix = attentive_work_in_class(model, blimp_structure, root_dir="data/BLIMP_auxillary/", metric="rtd")
#np.save('auxBERT_rtd.dat', smallBERT_rtd_matrix)

In [ ]:
smallBERT_h0s_matrix = attentive_work_in_class(model, blimp_structure, root_dir="data/BLIMP_auxillary/", metric="h0s")

# Head ensamble selection

In [60]:
import heapq
import copy

Auxillary step: expand matrices to be able to work with Rule II the same way as with Rule I

In [61]:
full_matrix = np.load('SciWork/test_04_sentence_pairs/full_matrix_cased.dat.npy')
generated_matrix = np.load('SciWork/test_04_sentence_pairs/generated/smoll_matrix_cased.dat.npy')

full_matrix = np.hstack([full_matrix, 1- full_matrix])
generated_matrix = np.hstack([generated_matrix, 1- generated_matrix])

(67000, 288) (6700, 288)
(67000, 288)


In [ ]:
full_matrix = np.hstack([fullBERT_rtd_matrix, 1- fullBERT_rtd_matrix])
generated_matrix = np.hstack([smallBERT_rtd_matrix, 1- smallBERT_rtd_matrix])

print(full_matrix.shape, generated_matrix.shape)
print(full_matrix.shape)

Function for selecting best ensamble of heads

In [78]:
def calc_proc(full_matrix, generated_matrix, phenomenons, max_len=51, max_queue_size=40):
  # Parameters:
  #           full_matrix      - dataset to score on (a.k.a. test data)
  #           generated_matrix - dataset to select ensembles on (a.k.a. train data)
  #           phenomenons      - dictionary structure of the dataset
  #           max_len          - maximal number of heads in the ensemble
  #           max_queue_size   - regulates the `width' of the beam search


  # Output: 
  #           Dictionary with keys <acc> - Accuracy of the best ensemble
  #                                <Rule I vertices>  - List of heads as pairs (# of layer, # of head within layer) that are used with Rule I in the best ensemble
  #                                <Rule II vertices> - List of heads as pairs (# of layer, # of head within layer) that are used with Rule II in the best ensemble

  small_matrix = generated_matrix

  entities = [ [np.mean(small_matrix[:, x] > 0.5), -x, [x]] for x in range(288)]
  cnt = 0
  best_score = -1
  best_vertex_list = []
  print("Length 1 --- calculations completed")
  for i in range(max_len // 2):
      h = []
      for elem in entities:
        for new_vert in range(288):
          for new_vert_2 in range(new_vert):
            if new_vert in elem[2] or new_vert_2 in elem[2] or new_vert - new_vert_2 == 144:
              continue
          
            msk = (np.sum(small_matrix[:, elem[2]], axis=1) + small_matrix[:, new_vert] + small_matrix[:, new_vert_2]) / (len(elem[2]) + 2)
            sum = np.mean(msk > 0.5)
            lst = elem[2] + [new_vert, new_vert_2]

            if sum >= elem[0]:
              if len(h) >= max_queue_size:
                heapq.heapreplace(h, [sum, cnt, lst])
              else:
                heapq.heappush(h, [sum, cnt, lst])

              cnt += 1

      entities = copy.deepcopy(sorted(h))

      if len(entities) == 0:
        continue
      best = sorted(entities)[-1]

      sm = np.zeros(full_matrix.shape[0])
      vert_list = []

      for vert in best[2]:
        sm += full_matrix[: , vert]
        vert_list.append((vert // 12, vert % 12))

      sm /= len(best[2])
      sm = (sm > 0.5).astype(int)

      if best_score >= 100 * np.mean(sm):
        break
      else:
        best_score =  100 * np.mean(sm)
        best_vertex_list = vert_list
      print("Length", i * 2 + 3, " --- calculations completed")

  result_dict = {'acc' : best_score, 'Rule I vertices' : [], 'Rule II vertices' : []}
  for vert in vert_list:
    if vert[0] < 12:
      result_dict['Rule I vertices'].append(vert)
    else:
      vert[0] -= 12
      result_dict['Rule II vertices'].append(vert)

  return result_dict

A simple example:

In [80]:
result_dict = calc_proc(full_matrix, generated_matrix, blimp_structure, max_len=3, max_queue_size=2)
print("Results: ", result_dict)

Length 1 --- calculations completed
Length 3  --- calculations completed
Results:  {'acc': 55.159701492537316, 'Rule I vertices': [(0, 7), (0, 9), (0, 1)], 'Rule II vertices': []}


Setup as it was used in our work:

In [ ]:
calc_proc(full_matrix, generated_matrix, blimp_structure, max_len=51, max_queue_size=40)